# Subword Tokenization Demo: BPE & WordPiece (from scratch)

This notebook demonstrates **Byte Pair Encoding (BPE)** and a simplified **WordPiece** tokenization flow from scratch.

These are educational implementations for clarity.

In [1]:
# --- Utilities ---
from collections import Counter
import re

def simple_preprocess(text):
    # Lowercase and split on non-alphanumerics (keep apostrophes)
    text = text.lower()
    text = re.sub(r"[^a-z0-9']+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()


## Part 1 — Byte Pair Encoding (BPE)
Start from characters; repeatedly merge the most frequent adjacent pair.

BPE builds words the same way LEGO bricks build houses — by repeatedly gluing the most common pieces together.

In [2]:
# --- BPE Implementation ---
def get_pair_frequencies(tokenized_words):
    pairs = Counter()
    for symbols in tokenized_words:
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i+1])] += 1
    return pairs

def merge_pair(tokenized_words, pair):
    a, b = pair
    merged_token = a + '·' + b  # visual joiner
    new_words = []
    for symbols in tokenized_words:
        i = 0
        new_symbols = []
        while i < len(symbols):
            if i < len(symbols) - 1 and symbols[i] == a and symbols[i+1] == b:
                new_symbols.append(merged_token)
                i += 2
            else:
                new_symbols.append(symbols[i])
                i += 1
        new_words.append(new_symbols)
    return new_words

def learn_bpe(corpus, target_vocab_size=60, verbose=True):
    words = []
    for line in corpus:
        words.extend(simple_preprocess(line))
    tokenized_words = [list(w) + ['</w>'] for w in words]
    vocab = set(sym for w in tokenized_words for sym in w)
    merges = []
    if verbose:
        print(f'Initial vocab size: {len(vocab)}')
        print('Sample word:', tokenized_words[0])
    while len(vocab) < target_vocab_size:
        pair_freq = get_pair_frequencies(tokenized_words)
        if not pair_freq:
            break
        best_pair, best_freq = pair_freq.most_common(1)[0]
        tokenized_words = merge_pair(tokenized_words, best_pair)
        merged_token = best_pair[0] + '·' + best_pair[1]
        merges.append(best_pair)
        vocab.add(merged_token)
        if verbose:
            print(f"Merge {len(merges):>3}: {best_pair} -> '{merged_token}' (freq={best_freq}) | vocab={len(vocab)}")
    return merges, vocab, tokenized_words

def bpe_tokenize(word, merges):
    symbols = list(word) + ['</w>']
    for a, b in merges:
        i = 0
        merged = []
        while i < len(symbols):
            if i < len(symbols) - 1 and symbols[i] == a and symbols[i+1] == b:
                merged.append(a + '·' + b)
                i += 2
            else:
                merged.append(symbols[i])
                i += 1
        symbols = merged
    return symbols


In [3]:
# --- BPE Demo ---
corpus = [
    'pizza is tasty',
    'pizzazz is flashy',
    'unbelievable flavors of pizzas',
    'i love pineapple pizza',
    'cheese on pizza is great',
    'pizzerias serve pizza',
]
merges, vocab, tokenized_words = learn_bpe(corpus, target_vocab_size=60, verbose=True)
print('\nFirst 15 merges:')
for i, m in enumerate(merges[:15], 1):
    print(f'{i:>2}. {m}')
print('\nBPE tokenize examples:')
for w in ['pizza', 'pizzazz', 'pineapple', 'unbelievable']:
    print(w, '->', bpe_tokenize(w, merges))


Initial vocab size: 20
Sample word: ['p', 'i', 'z', 'z', 'a', '</w>']
Merge   1: ('p', 'i') -> 'p·i' (freq=8) | vocab=21
Merge   2: ('z', 'z') -> 'z·z' (freq=8) | vocab=22
Merge   3: ('p·i', 'z·z') -> 'p·i·z·z' (freq=7) | vocab=23
Merge   4: ('p·i·z·z', 'a') -> 'p·i·z·z·a' (freq=6) | vocab=24
Merge   5: ('s', '</w>') -> 's·</w>' (freq=6) | vocab=25
Merge   6: ('e', '</w>') -> 'e·</w>' (freq=5) | vocab=26
Merge   7: ('p·i·z·z·a', '</w>') -> 'p·i·z·z·a·</w>' (freq=4) | vocab=27
Merge   8: ('i', 's·</w>') -> 'i·s·</w>' (freq=3) | vocab=28
Merge   9: ('a', 's') -> 'a·s' (freq=2) | vocab=29
Merge  10: ('y', '</w>') -> 'y·</w>' (freq=2) | vocab=30
Merge  11: ('f', 'l') -> 'f·l' (freq=2) | vocab=31
Merge  12: ('l', 'e·</w>') -> 'l·e·</w>' (freq=2) | vocab=32
Merge  13: ('v', 'e·</w>') -> 'v·e·</w>' (freq=2) | vocab=33
Merge  14: ('e', 'a') -> 'e·a' (freq=2) | vocab=34
Merge  15: ('e', 'r') -> 'e·r' (freq=2) | vocab=35
Merge  16: ('t', 'a·s') -> 't·a·s' (freq=1) | vocab=36
Merge  17: ('t·a·s',

## Part 2 — WordPiece (Simplified)
We'll build a toy vocab by substring frequency and tokenize with greedy longest-match (MaxMatch). We use the '##' prefix for continuation pieces.

In [6]:
# --- WordPiece (Simplified) ---
from collections import defaultdict

def all_substrings(word, max_len=8):
    subs = []
    n = len(word)
    for i in range(n):
        for j in range(i+1, min(n, i+max_len) + 1):
            subs.append(word[i:j])
    return subs

def build_wordpiece_vocab(corpus, target_vocab_size=120, max_sub_len=8, verbose=True):
    words = []
    for line in corpus:
        words.extend(simple_preprocess(line))
    sub_freq = Counter()
    char_set = set()
    for w in words:
        char_set.update(w)
        for s in all_substrings(w, max_len=max_sub_len):
            sub_freq[s] += 1
    vocab = set(list(char_set))
    specials = {'[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'}
    vocab |= specials
    ranked = sorted(sub_freq.items(), key=lambda kv: (kv[1], len(kv[0])), reverse=True)
    if verbose:
        print(f'Initial vocab size (chars+specials): {len(vocab)}')
        print('Top 10 substrings by freq:', ranked[:10])
    for s, freq in ranked:
        if len(vocab) >= target_vocab_size:
            break
        if s not in vocab:
            vocab.add(s)
        if len(vocab) < target_vocab_size and len(s) > 1:
            vocab.add('##' + s)
    return vocab

def wordpiece_tokenize(word, vocab, unk_token='[UNK]'):
    tokens = []
    start = 0
    while start < len(word):
        end = len(word)
        cur_sub = None
        while start < end:
            piece = word[start:end] if len(tokens) == 0 else '##' + word[start:end]
            if piece in vocab:
                cur_sub = piece
                break
            end -= 1
        if cur_sub is None:
            return [unk_token]
        tokens.append(cur_sub)
        advance = len(cur_sub) if len(tokens) == 1 else len(cur_sub) - 2
        start += advance
    return tokens


In [7]:
# --- WordPiece Demo ---
vocab_wp = build_wordpiece_vocab(corpus, target_vocab_size=120, max_sub_len=8, verbose=True)
examples = ['pizza', 'pizzazz', 'pineapple', 'unbelievable', 'pizzerias', 'cheesiness']
for w in examples:
    print(f"{w:>13} -> {wordpiece_tokenize(w, vocab_wp)}")


Initial vocab size (chars+specials): 24
Top 10 substrings by freq: [('z', 16), ('i', 14), ('a', 13), ('e', 13), ('p', 10), ('s', 10), ('pi', 8), ('zz', 8), ('pizz', 7), ('piz', 7)]
        pizza -> ['pizza']
      pizzazz -> ['pizzazz']
    pineapple -> ['[UNK]']
 unbelievable -> ['[UNK]']
    pizzerias -> ['[UNK]']
   cheesiness -> ['[UNK]']
